In [ ]:
import os
import re
import json
import warnings
import anthropic
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI

from langchain.schema import Document
from langchain_community.chat_models import ChatOllama, ChatOpenAI

from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_upstage import UpstageEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = anthropic_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

In [2]:
class Args:
    retrieval_debug = False
    llm_model = "ollama"
    
    src_lang = "ko"
    if src_lang == "en":
        eval_file_path = "../dataset/eval.jsonl" ## "../dataset/en_eval.jsonl" --> 성능이 별로임.
        doc_file_path = "../dataset/en_4.0_document.jsonl" ## "../dataset/processed_documents.jsonl"
    else:
        eval_file_path = "../dataset/eval.jsonl"
        doc_file_path = "../dataset/processed_documents.jsonl"

    output_path = "./outputs/output.csv"

    ## sparse or dense or ensemble
    doc_method = "dense"
    encoder_method = "upstage" ## huggingface, upstage, openai
    retriever_weights = [0.3, 0.7] ## [sparse, dense]

    ## HuggingFace
    hf_model_name = "intfloat/multilingual-e5-large-instruct"
    model_kwargs = {"device": "cuda:0"}
    encode_kwargs = {"normalize_embeddings": False,
                     "clean_up_tokenization_spaces": True}
    
    ## Upstage
    upstage_model_name = "solar-embedding-1-large-passage"
    faiss_index_file = "./index_files/upstage-faiss.npy"
    
    ## OpenAI
    openai_model_name = "text-embedding-3-large"

    ## chunking
    chunking = True
    chunk_method = "recursive" ## recursive, semantic
    semantic_chunk_method = "upstage"
    chunk_size = 100
    chunk_overlap = 50

    ## query expension
    query_expansion = False

    ## query ensemble
    query_ensemble = False  # 쿼리 앙상블 수행 여부
    # 앙상블에 사용할 모델
    ensemble_models = [
        # {'type': 'hf', 'name': ""},
        # {'type': 'hf', 'name': "nlpai-lab/KoE5"},
        # {'type': 'hf', 'name': "BAAI/bge-large-en-v1.5",
        # {'type': 'hf', 'name': "intfloat/multilingual-e5-large"},
        # {'type': 'upstage', 'name': "solar-embedding-1-large-query"},
        # {'type': 'hf', 'name': "sentence-transformers/all-MiniLM-L6-v2"},
    ]
    ensemble_weights = [1]  # 각각의 모델 가중치 설정

    ## reranker
    rerank = False
    reranker_name = "BAAI/bge-reranker-v2-m3"  ## "BAAI/bge-reranker-large"

In [3]:
def load_ollama_encoder(model_name):
    encoder = OllamaEmbeddings(model_name)

    return encoder

def load_upstage_encoder(model_name):
    encoder = UpstageEmbeddings(model=model_name)

    return encoder

def load_openai_encoder(model_name):
    encoder = OpenAIEmbeddings(model=model_name)

    return encoder

def load_hf_encoder(model_name, model_kwargs, encode_kwargs):
    encoder = HuggingFaceEmbeddings(model_name=model_name,
                                    model_kwargs=model_kwargs,
                                    encode_kwargs=encode_kwargs)
    
    return encoder

def load_hf_reranker(model_name, retriever):
    reranker = HuggingFaceCrossEncoder(model_name=model_name)
    compressor = CrossEncoderReranker(model=reranker, top_n=3)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever.as_retriever(search_kwargs={"k": 10}))

    return compression_retriever

In [4]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]
    
def load_document(path='../dataset/documents.jsonl'):
    raw_documents = load_jsonl(path)

    documents = []
    for doc in raw_documents:
        doc_id = doc['docid']
        content = doc['content']
        documents.append(Document(page_content=content, metadata={"docid": doc_id}))

    return documents

def chunking(args, documents):
    if args.chunk_method == "recursive":
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=args.chunk_size,
            chunk_overlap=args.chunk_overlap,
            length_function=len,
            is_separator_regex=False
        )
    elif args.chunk_method == "semantic":
        if args.semantic_chunk_method == "huggingface":
            encoder = load_hf_encoder(args.hf_model_name, args.model_kwargs, args.encode_kwargs)
        elif args.semantic_chunk_method == "upstage":
            encoder = load_upstage_encoder(args.upstage_model_name)
        elif args.semantic_chunk_method == "openai":
            encoder = load_openai_encoder(args.openai_model_name)

        text_splitter = SemanticChunker(encoder)

    return text_splitter.split_documents(documents)

In [ ]:
args = Args()

total_documents = load_document(path=args.doc_file_path)
print(len(total_documents))

chunk_documents = chunking(args, total_documents)
print(len(chunk_documents))

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = args.chunk_size,
    chunk_overlap  = args.chunk_overlap,
    length_function = len,
)

In [7]:
def ollama_contextual_retrieval(model):
    prompt_context3 = (
    """
    <document>
    {DOCUMENT}
    </document> 

    전체 문서 내에 배치하려는 청크는 다음과 같습니다.

    <chunk> 
    {CHUNK}
    </chunk>
    
    이 청크가 전체 문서에서 어떤 맥락에 속하는지 설명하는 간결한 문맥을 한국어로 작성하세요. 답변은 이 청크에 관한 짧고 구체적인 배경 설명을 포함해야 하며, 청크가 문서의 어느 부분에서 나온 것인지에 대한 정보를 제공해야 합니다.
    
    입력 예시:
        회사의 매출이 전 분기 대비 3% 증가했습니다.

    주어진 청크 예시에서는 '회사'가 어떤 회사를 말하는 것인지, '전 분기'가 정확하게 몇년도 몇분기에 대한 것인지 정보가 포함되어 있지 않습니다. 따라서 당신은 아래 출력 예시처럼 입력되는 청크를 읽고 정보검색에 유용하도록 더 명확한 청크로 재구성해야합니다.

    출력 예시: 
        이 청크는 2023년 2분기에 ACME 회사의 실적을 다룬 SEC 보고서에서 발췌되었습니다. 이전 분기의 수익은 3억 1천 4백만 달러였으며, 회사의 수익은 이전 분기 대비 3% 증가했습니다.
    """
    )

    prompt3 = ChatPromptTemplate.from_template(prompt_context3)
    chain3 = prompt3 | model | StrOutputParser()

    return chain3

In [10]:
model = ChatOllama(model="eeve-10.8b-q8:latest", temperature=0.2)
chain3 = ollama_contextual_retrieval(model)

In [ ]:
with open('../dataset/ollama_contextual_retrieval_documents.jsonl', 'w', encoding='utf-8') as f:
    # for document in tqdm(documents):
    for i, document in enumerate(total_documents):
        print("=" * 50)
        print(i)
        print(f"docid : {document.metadata['docid']}")
        print(f"page_content : {document.page_content}")

        chunks = text_splitter.split_text(document.page_content)
        print(f"num of chunks : {len(chunks)}\n")

        for idx, chunk in enumerate(chunks, start=1):
            print(f"chunk {idx} : {chunk}")
            result = chain3.invoke({"DOCUMENT" : document.page_content, "CHUNK" : chunk})
            print(f"output : {result}\n")

            result_with_id = {
                "docid": document.metadata['docid'],
                "content": f"{chunk}\n\n{result}"
            }
            f.write(json.dumps(result_with_id, ensure_ascii=False) + '\n')

In [22]:
# client = OpenAI()
# model = "gpt-3.5-turbo" ## "gpt-4o"

client = OpenAI(
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)
model = "solar-pro"

In [27]:
def gpt_contextual_retrieval(document, chunk, model: str, client: OpenAI):
    prompt = (
    """
    <document>
    {DOCUMENT}
    </document> 

    전체 문서 내에 배치하려는 청크는 다음과 같습니다.

    <chunk> 
    {CHUNK}
    </chunk>
    
    이 청크가 전체 문서에서 어떤 맥락에 속하는지 설명하는 간결한 문맥을 한국어로 작성하세요. 답변은 이 청크에 관한 짧고 구체적인 배경 설명을 포함해야 하며, 청크가 문서의 어느 부분에서 나온 것인지에 대한 정보를 제공해야 합니다.
    
    입력 예시:
        회사의 매출이 전 분기 대비 3% 증가했습니다.

    예시에 대한 설명:
        주어진 청크 예시에서는 '회사'가 어떤 회사를 말하는 것인지, '전 분기'가 정확하게 몇년도 몇분기에 대한 것인지 정보가 포함되어 있지 않습니다. 따라서 당신은 아래 출력 예시처럼 입력되는 청크를 읽고 정보검색에 유용하도록 더 명확한 청크로 재구성해야합니다.

    출력 예시: 
        이 청크는 2023년 2분기에 ACME 회사의 실적을 다룬 SEC 보고서에서 발췌되었습니다. 이전 분기의 수익은 3억 1천 4백만 달러였으며, 회사의 수익은 이전 분기 대비 3% 증가했습니다.
    """
    ).format(DOCUMENT=document, CHUNK=chunk)

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )
    
    response = completion.choices[0].message.content
    return response


In [28]:
with open('../dataset/gpt_contextual_retrieval_documents.jsonl', 'w', encoding='utf-8') as f:
    # for document in tqdm(documents):
    for idx, document in enumerate(total_documents):
        print("=" * 50)
        print(idx)
        print(f"docid : {document.metadata['docid']}")
        print(f"page_content : {document.page_content}")

        chunks = text_splitter.split_text(document.page_content)
        print(f"num of chunks : {len(chunks)}\n")

        for idx, chunk in enumerate(chunks, start=1):
            print(f"chunk {idx} : {chunk}")
            result = gpt_contextual_retrieval(document, chunk, model, client)
            print(f"output : {result}\n")

            result_with_id = {
                "docid": document.metadata['docid'],
                "content": f"{chunk}\n\n{result}"
            }
            f.write(json.dumps(result_with_id, ensure_ascii=False) + '\n')

0
docid : 42508ee0-c543-4338-878e-d98c6babee66
page_content : 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.
num of chunks : 7

chunk 1 : 건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간
output : 
이 청크는 건강과 건강 유지에 관한 문서의 처음 부분에서 나온 것입니다. 청크는 에너지 균형이 무엇인지, 건강한 사람이 에너지 균형을 달성하는 데 걸리는 일반적인 기간, 에너지 균형을 달성하는 데 필요한 조절 방법에 대한 기본 정보를 제공합니다. 청크는 에너지 균형이 에너지 섭취와 에너지 소비의 수학적 동등성을 의미하고, 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성하는 것이 중요하고, 에너지 균형을 유지하기 위해 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 한다는 내용을 설명합니다.

chunk 2 : 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지
output 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4124 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [13]:
client = anthropic.Anthropic(api_key=anthropic_api_key)

In [16]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
전체 문서 내에 배치하려는 청크는 다음과 같습니다.
<chunk>
{chunk_content}
</chunk>

이 청크가 전체 문서에서 어떤 맥락에 속하는지 설명하는 간결한 문맥을 한국어로 작성하세요. 답변은 이 청크에 관한 짧고 구체적인 배경 설명을 포함해야 하며, 청크가 문서의 어느 부분에서 나온 것인지에 대한 정보를 제공해야 합니다.

    입력 예시:
        회사의 매출이 전 분기 대비 3% 증가했습니다.

    주어진 청크 예시에서는 '회사'가 어떤 회사를 말하는 것인지, '전 분기'가 정확하게 몇년도 몇분기에 대한 것인지 정보가 포함되어 있지 않습니다. 따라서 당신은 아래 출력 예시처럼 입력되는 청크를 읽고 정보검색에 유용하도록 더 명확한 청크로 재구성해야합니다.

    출력 예시: 
        이 청크는 2023년 2분기에 ACME 회사의 실적을 다룬 SEC 보고서에서 발췌되었습니다. 이전 분기의 수익은 3억 1천 4백만 달러였으며, 회사의 수익은 이전 분기 대비 3% 증가했습니다.
"""

def situate_context(doc: str, chunk: str) -> str:
    response = client.beta.prompt_caching.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.0,
        messages=[
            {
                "role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc),
                        "cache_control": {"type": "ephemeral"} #we will make use of prompt caching for the full documents
                    },
                    {
                        "type": "text",
                        "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk),
                    }
                ]
            }
        ],
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )
    return response

In [ ]:
with open('../dataset/antropic_contextual_retrieval_documents.jsonl', 'w', encoding='utf-8') as f:
    # for document in tqdm(documents):
    for idx, document in enumerate(total_documents):
        print("=" * 50)
        print(idx)
        print(f"docid : {document.metadata['docid']}")
        print(f"page_content : {document.page_content}")

        chunks = text_splitter.split_text(document.page_content)
        print(f"num of chunks : {len(chunks)}\n")

        for idx, chunk in enumerate(chunks, start=1):
            print(f"chunk {idx} : {chunk}")
            result = situate_context(document, chunk)
            print(f"output : {result.content[0].text}\n")

            result_with_id = {
                "docid": document.metadata['docid'],
                "content": f"{chunk}\n\n{result.content[0].text}"
            }
            f.write(json.dumps(result_with_id, ensure_ascii=False) + '\n')